In [ ]:
import numpy as np
import glob
from pathlib import Path
import pickle
import gc
import tensorflow as tf
from keras.models import Model
from ase.io.lammpsdata import read_lammps_data as rld
from pymatgen.io.ase import AseAtomsAdaptor
from megnet.utils.models import load_model

def structures_generator():
    files = glob.glob('processed/data.*.txt')
    for file in files:
        data = rld(file, style='charge')
        structure = AseAtomsAdaptor.get_structure(data)
        yield structure




def get_features(models, energies):
    with tf.device('/GPU:0'):
        for mg_model in models:
            model = load_model(mg_model)
            model.layers.pop()
            model2 = Model(model.input, model.layers[-2].output)
            model.model = model2

            features_list = []
            structures_gen = structures_generator()
            struct_count = 0
            for structure, energy in zip(structures_gen, energies):
                features = model.predict_structure(structure)
                features_list.append(np.append(features, energy))
                struct_count += 1
                print(struct_count)
                del structure
                gc.collect()

            features_array = np.vstack(features_list)
            np.save('features/' + mg_model + '_data.npy', features_array)




def main():
    models = ['Eform_MP_2019', 'Efermi_MP_2019', 'Bandgap_MP_2018', 'logK_MP_2019', 'logG_MP_2019']
    processed_dir = Path('processed')
    features_dir = Path('features')

    # Create 'features' directory if it doesn't exist
    features_dir.mkdir(parents=True, exist_ok=True)

    with open('processed_energies_list.pkl', 'rb') as f:
        md = pickle.load(f)

    get_features(models, md)

if __name__ == "__main__":
    main()